In [1]:
# put all imports in one sell to avoid running all cells to continue programming
%matplotlib inline
import matplotlib.pyplot as plt
import sys  
import pandas as pd
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/features')
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/data')
from sklearn.impute import SimpleImputer
from dataCleaner import data_cleaning
from columnsSpliter import get_columns
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import sklearn
from load_data import internal_load
# housing_train= internal_load("C:/Users/Hossein Karimi/Documents/pprojects/housing_hw/data/external/train.csv")
data = internal_load("C:/Users/h.karimi/Documents/pprojects/housing_hw/data/interim/housing_train_cleaned.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 0 to 1167
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotFrontage    1168 non-null   float64
 1   LotArea        1168 non-null   int64  
 2   YearBuilt      1168 non-null   int64  
 3   YearRemodAdd   1168 non-null   int64  
 4   MasVnrArea     1168 non-null   float64
 5   BsmtFinSF1     1168 non-null   int64  
 6   BsmtFinSF2     1168 non-null   int64  
 7   BsmtUnfSF      1168 non-null   int64  
 8   TotalBsmtSF    1168 non-null   int64  
 9   1stFlrSF       1168 non-null   int64  
 10  2ndFlrSF       1168 non-null   int64  
 11  LowQualFinSF   1168 non-null   int64  
 12  GrLivArea      1168 non-null   int64  
 13  BsmtFullBath   1168 non-null   int64  
 14  BsmtHalfBath   1168 non-null   int64  
 15  FullBath       1168 non-null   int64  
 16  HalfBath       1168 non-null   int64  
 17  BedroomAbvGr   1168 non-null   int64  
 18  KitchenA

In [2]:
# #spectral categorical data points to those data that somthing increases or decreases in their values
# # for example "Regularity" in LotShape atttribute; IR1 and IR2 are diffrent values becuase of the amounts of Regularity
# # we should not deal with thease types of data like those don't have any spectral concept between their values
# # for example "MSZoning" values doesn't have common spectral concept
# def spectral_cat_attrib_to_num (data,attrib,specs):  
#     data_copy = data.copy()
#     l = len(data_copy[attrib])
#     i = 0
#     while i != l:
#         print(data_copy.at[i,attrib])
#         data_copy.at[i,attrib] = specs[data_copy.at[i,attrib]]
#         i+=1
#     new_data = data_copy.copy()
#     return new_data
# # this function gets dataset and an attrib and a dict including value of attributes as the keys and the ...
# #...spectral order of value -that it is an integer - as values, and then return the whole dataset with spectral 
# #...values replaced with an integer showing spectral order
# # we call the above dict, attribiute's "specs_dict"
# # the below function returns a "specs_dict" for input attribute
# # this function outputs is one of th "spectral_cat_data_to_num" function inputs
# def specs_maker_for_spectral_attribs(values):
# #   values = attrib.unique()
#     spec_dict = {}
#     i=0
#     while i <= (len(values)-1):
#         spec_dict[values[i]] = i
#         i+=1
#     return spec_dict

# # the "spec_attribs_orders" in below is a dict with data spectral attribiutes as keys, and their ordered unique values list as values
# def spectral_cat_data_to_num(data,spec_attribs_orders):
#     data_copy = data.copy()
#     for key in spec_attribs_orders.keys():
#         specs_dict = {}
#         specs_dict = specs_maker_for_spectral_attribs(spec_attribs_orders[key])
#         data_copy = spectral_cat_attrib_to_num(data_copy,key,specs_dict)
#         specs_dict.clear()
#     new_data = data_copy.copy()
#     return new_data

In [3]:
# # as if you know, there are some other attribiutes didn't covered above. i.e. condition1, condition 2
# # these kinds of attributes have same categories and meaining.
# # as we used the one-hat-encoder for categorical attribs and in this method, just one cat-columns is 1, 
# # I decided to select two cat-columns insted one to set value for the record 1
# # so build another method using the original one-hat-encoder to make this happen

# def second_cat_data_to_num(data,attrib_1,attrib_2):
#     data_copy = data.copy()
#     while i <= len(attrib_2):
#         item = attrib_2[i]
#         data_copy.at[i,attrib_1 + " " + item] = 1
#         i += 1
#     return data_copy

In [4]:
# from dataPreparation import num_data_null_handler,binary_cat_to_num,cat_data_to_num
# data_handeled_null_num = num_data_null_handler(data).copy()
# data_numed_cats = cat_data_to_num(data_handeled_null_num, ["MSZoning","Alley"]).copy()
# data_numed_cats_bins = binary_cat_to_num(data_numed_cats, "Street", "Pave").copy()
# data_numed_cats_bins_specCats = spectral_cat_data_to_num(data_numed_cats_bins,the_spec_attribs_orders)
# # # LotShape_specs_categories = ["Reg","IR1","IR2","IR3"]
# # LotShape_specs= specs_maker_for_spectral_attribs(data["LotShape"])
# # cat_data_transformed_to_num = spectral_cat_data_to_num(cat_data_transformed_to_num,"LotShape",LotShape_specs)
# # cat_data_transformed_to_num["LotShape"].unique()

In [56]:
def cat_column_encoder(data, cat_columns):
    data_copy = data.copy()
    new_data = data_copy.copy()
    _cat_encoder = OneHotEncoder() 
    for col in cat_columns:
        new_data = new_data.drop(col, axis = 1) # isolate the non categorical data
    for col in cat_columns:
        encoded_col = _cat_encoder.fit_transform(data_copy[col].to_numpy().reshape(-1, 1))
        cols =[col + ": " +str(s) for s in _cat_encoder.categories_[0]]
        encoded_col_df = pd.DataFrame(encoded_col.toarray(), columns = cols)
        new_data = pd.concat([new_data, encoded_col_df], axis=1)
        encoded_col_df = pd.DataFrame({})
    return new_data

In [57]:
new_data = cat_column_encoder(data,get_columns('cat'))

In [60]:
new_data

,Unnamed: 0,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSSubClass: 20,MSSubClass: 30,MSSubClass: 40,MSSubClass: 45,MSSubClass: 50,MSSubClass: 60,MSSubClass: 70,MSSubClass: 75,MSSubClass: 80,MSSubClass: 85,MSSubClass: 90,MSSubClass: 120,MSSubClass: 160,MSSubClass: 180,MSSubClass: 190,MSZoning: c (all),MSZoning: fv,MSZoning: rh,MSZoning: rl,MSZoning: rm,Street: grvl,Street: pave,Alley: grvl,Alley: na,Alley: pave,LotShape: ir1,LotShape: ir2,LotShape: ir3,LotShape: reg,LandContour: bnk,LandContour: hls,LandContour: low,LandContour: lvl,Utilities: allpub,Utilities: nosewa,LotConfig: corner,LotConfig: culdsac,LotConfig: fr2,LotConfig: fr3,LotConfig: inside,LandSlope: gtl,LandSlope: mod,LandSlope: sev,Neighborhood: blmngtn,Neighborhood: blueste,Neighborhood: brdale,Neighborhood: brkside,Neighborhood: clearcr,Neighborhood: collgcr,Neighborhood: crawfor,Neighborhood: edwards,Neighborhood: gilbert,Neighborhood: idotrr,Neighborhood: meadowv,Neighborhood: mitchel,Neighborhood: names,Neighborhood: noridge,Neighborhood: npkvill,Neighborhood: nridght,Neighborhood: nwames,Neighborhood: oldtown,Neighborhood: sawyer,Neighborhood: sawyerw,Neighborhood: somerst,Neighborhood: stonebr,Neighborhood: swisu,Neighborhood: timber,Neighborhood: veenker,Condition1: artery,Condition1: feedr,Condition1: norm,Condition1: posa,Condition1: posn,Condition1: rrae,Condition1: rran,Condition1: rrne,Condition1: rrnn,Condition2: artery,Condition2: feedr,Condition2: norm,Condition2: posn,Condition2: rrae,Condition2: rrnn,BldgType: 1fam,BldgType: 2fmcon,BldgType: duplex,BldgType: twnhs,BldgType: twnhse,HouseStyle: 1.5fin,HouseStyle: 1.5unf,HouseStyle: 1story,HouseStyle: 2.5fin,HouseStyle: 2.5unf,HouseStyle: 2story,HouseStyle: sfoyer,HouseStyle: slvl,OverallQual: 1,OverallQual: 2,OverallQual: 3,OverallQual: 4,OverallQual: 5,OverallQual: 6,OverallQual: 7,OverallQual: 8,OverallQual: 9,OverallQual: 10,OverallCond: 1,OverallCond: 2,OverallCond: 3,OverallCond: 4,OverallCond: 5,OverallCond: 6,OverallCond: 7,OverallCond: 8,OverallCond: 9,RoofStyle: flat,RoofStyle: gable,RoofStyle: gambrel,RoofStyle: hip,RoofStyle: mansard,RoofStyle: shed,RoofMatl: clytile,RoofMatl: compshg,RoofMatl: membran,RoofMatl: metal,RoofMatl: tar&grv,RoofMatl: wdshake,RoofMatl: wdshngl,Exterior1st: asbshng,Exterior1st: brkcomm,Exterior1st: brkface,Exterior1st: cblock,Exterior1st: cemntbd,Exterior1st: hdboard,Exterior1st: imstucc,Exterior1st: metalsd,Exterior1st: plywood,Exterior1st: stone,Exterior1st: stucco,Exterior1st: vinylsd,Exterior1st: wd sdng,Exterior1st: wdshing,Exterior2nd: asbshng,Exterior2nd: asphshn,Exterior2nd: brkcomm,Exterior2nd: brkface,Exterior2nd: cblock,Exterior2nd: cemntbd,Exterior2nd: hdboard,Exterior2nd: imstucc,Exterior2nd: metalsd,Exterior2nd: other,Exterior2nd: plywood,Exterior2nd: stone,Exterior2nd: stucco,Exterior2nd: vinylsd,Exterior2nd: wd sdng,Exterior2nd: wdshing,MasVnrType: brkcmn,MasVnrType: brkface,MasVnrType: na,MasVnrType: none,MasVnrType: stone,ExterQual: ex,ExterQual: fa,ExterQual: gd,ExterQual: ta,ExterCond: ex,ExterCond: fa,ExterCond: gd,ExterCond: ta,Foundation: brktil,Foundation: cblock,Foundation: pconc,Foundation: slab,Foundation: stone,Foundation: wood,BsmtQual: ex,BsmtQual: fa,BsmtQual: gd,BsmtQual: na,BsmtQual: ta,BsmtCond: fa,BsmtCond: gd,BsmtCond: na,BsmtCond: po,BsmtCond: ta,BsmtExposure: av,BsmtExposure: gd,BsmtExposure: mn,BsmtExposure: na,BsmtExposure: no,BsmtFinType1: alq,BsmtFinType1: blq,BsmtFinType1: glq,BsmtFinType1: lwq,BsmtFinType1: na,BsmtFinType1: rec,BsmtFinType1: unf,BsmtFinType2: alq,BsmtFinType2: blq,BsmtFinType2: glq,BsmtFinType2: lwq,BsmtFinType2: na,BsmtFinType2: rec,BsmtFinType2: unf,Heating: floor,Heating: gasa,Heating: gasw

In [62]:
pd.set_option('display.max_columns', None)

In [67]:
new_data.to_csv(r'C:/Users/h.karimi/Documents/pprojects/housing_hw/data/interim/housing_train_cleaned_nummed.csv')